# Snowflake ML Features Demo
## SQL Feature Engineering, FORECAST Baseline, and Feature Store

This notebook demonstrates **Snowflake-native ML capabilities** that complement the MMM training pipeline.

### What This Notebook Shows

| Feature | Description | When to Use |
|---------|-------------|-------------|
| **SQL Feature Engineering** | Fourier seasonality, trend in SQL views | Simple features, single model |
| **FORECAST Function** | Naive time-series baseline | Compare MMM vs. statistical forecast |
| **Feature Store** | Centralized feature management | Multiple models sharing features |

### Key Insight

For this MMM demo, **a SQL view handles feature engineering perfectly**. Feature Store adds value when:
- Multiple ML models share the same features
- You need point-in-time correctness for training vs. serving
- Feature governance and lineage is critical

---
**Prerequisites**: Run `01_mmm_training.ipynb` first to populate model results.

In [ ]:
# =============================================================================
# CELL 1: Setup and Connection
# =============================================================================

import pandas as pd
import numpy as np
from datetime import datetime

from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import functions as F

session = get_active_session()
print(f"Connected to: {session.get_current_database()}.{session.get_current_schema()}")

## Section 1: SQL-Computed Features

Instead of computing features in Python, we can push feature engineering to SQL. This:
- Scales to any data size (Snowflake handles compute)
- Ensures consistency between training and serving
- Reduces data transfer overhead

### Fourier Seasonality

Traditional approach: 52 dummy variables for week-of-year (wasteful, overfits)

Better approach: Fourier terms capture smooth seasonal patterns:
```
f(t) = a₀ + Σₖ [aₖ·cos(2πkt/52) + bₖ·sin(2πkt/52)]
```

We use k=1,2,3 giving us 6 features that capture annual, semi-annual, and quarterly patterns.

In [ ]:
# =============================================================================
# CELL 2: SQL Feature Engineering Example
# =============================================================================
# 
# This shows how Fourier seasonality and trend can be computed in SQL.
# =============================================================================

sql_feature_example = """
-- SQL-computed seasonality features example
SELECT 
    WEEK_START,
    WEEKOFYEAR(WEEK_START) AS WEEK_OF_YEAR,
    DATEDIFF('day', '2022-01-01', WEEK_START) / 365.25 AS TREND,
    
    -- Fourier Terms: Annual cycle (k=1)
    SIN(2 * PI() * 1 * WEEKOFYEAR(WEEK_START) / 52) AS SIN_1,
    COS(2 * PI() * 1 * WEEKOFYEAR(WEEK_START) / 52) AS COS_1,
    
    -- Fourier Terms: Semi-annual cycle (k=2)  
    SIN(2 * PI() * 2 * WEEKOFYEAR(WEEK_START) / 52) AS SIN_2,
    COS(2 * PI() * 2 * WEEKOFYEAR(WEEK_START) / 52) AS COS_2,
    
    -- B2B Fiscal Flags
    CASE WHEN MONTH(WEEK_START) BETWEEN 1 AND 3 THEN 1 ELSE 0 END AS Q1_FLAG,
    CASE WHEN MONTH(WEEK_START) BETWEEN 7 AND 9 THEN 1 ELSE 0 END AS Q3_FLAG
    
FROM (SELECT DATEADD('week', SEQ4(), '2022-01-01')::DATE AS WEEK_START 
      FROM TABLE(GENERATOR(ROWCOUNT => 104)))
LIMIT 10
"""

print("SQL Feature Engineering Example:")
print("="*60)
print(sql_feature_example)

# Execute and show results
df_features = session.sql(sql_feature_example).to_pandas()
print("\nSample Output:")
df_features.round(4)

## Section 2: FORECAST Baseline Comparison

Snowflake's `FORECAST` function provides a naive time-series baseline. This is useful to:
- Demonstrate that MMM outperforms pure pattern-matching
- Establish a "no marketing intelligence" baseline
- Show Snowflake's built-in ML capabilities

### Why MMM Should Outperform FORECAST

FORECAST just extrapolates historical patterns. It doesn't know:
- That a $500K LinkedIn campaign is launching next quarter
- That competitor SOV is increasing
- The causal relationship between spend and revenue

MMM captures these relationships explicitly.

In [ ]:
# =============================================================================
# CELL 3: Create FORECAST Baseline
# =============================================================================

def create_forecast_baseline(session):
    """Create a FORECAST model as a baseline for comparison."""
    
    print("\n" + "="*60)
    print("SNOWFLAKE FORECAST BASELINE")
    print("="*60)
    
    try:
        # Step 1: Create aggregated revenue view for forecasting
        session.sql("""
            CREATE OR REPLACE VIEW MMM.V_WEEKLY_REVENUE_FOR_FORECAST AS
            SELECT 
                WEEK_START,
                SUM(REVENUE) AS TOTAL_REVENUE
            FROM DIMENSIONAL.V_MMM_INPUT_WEEKLY
            WHERE WEEK_START IS NOT NULL AND REVENUE > 0
            GROUP BY WEEK_START
            ORDER BY WEEK_START
        """).collect()
        print("✓ Created revenue view for forecasting")
        
        # Step 2: Create FORECAST model
        session.sql("""
            CREATE OR REPLACE SNOWFLAKE.ML.FORECAST MMM.REVENUE_FORECAST_MODEL (
                INPUT_DATA => SYSTEM$REFERENCE('VIEW', 'MMM.V_WEEKLY_REVENUE_FOR_FORECAST'),
                TIMESTAMP_COLNAME => 'WEEK_START',
                TARGET_COLNAME => 'TOTAL_REVENUE'
            )
        """).collect()
        print("✓ Created FORECAST model: MMM.REVENUE_FORECAST_MODEL")
        
        # Step 3: Generate 13-week forecast and save to table
        session.sql("""
            CREATE OR REPLACE TABLE MMM.FORECAST_BASELINE AS
            SELECT 
                TS AS FORECAST_WEEK,
                FORECAST AS FORECAST_REVENUE,
                LOWER_BOUND AS FORECAST_LOWER_90,
                UPPER_BOUND AS FORECAST_UPPER_90,
                'SNOWFLAKE_FORECAST' AS MODEL_TYPE,
                CURRENT_TIMESTAMP() AS GENERATED_AT
            FROM TABLE(MMM.REVENUE_FORECAST_MODEL!FORECAST(
                FORECASTING_PERIODS => 13,
                CONFIG_OBJECT => {'prediction_interval': 0.9}
            ))
        """).collect()
        print("✓ Generated 13-week forecast to MMM.FORECAST_BASELINE")
        
        # Show forecast results
        df_forecast = session.table("MMM.FORECAST_BASELINE").to_pandas()
        print(f"\nForecast Preview (next 13 weeks):")
        print(df_forecast[['FORECAST_WEEK', 'FORECAST_REVENUE', 'FORECAST_LOWER_90', 'FORECAST_UPPER_90']].to_string(index=False))
        
        return True
        
    except Exception as e:
        print(f"\n⚠ FORECAST creation skipped: {str(e)}")
        print("  This may be due to:")
        print("  - ML Functions not enabled in your account")
        print("  - Insufficient data for time-series modeling")
        print("  - Missing privileges")
        return False

# Create forecast baseline
forecast_created = create_forecast_baseline(session)

## Section 3: Feature Store - When to Use It

### For This Demo: A SQL View Is Sufficient

We have:
- **One model** (MMM) using these features
- **Simple features** (Fourier terms, trend, flags)
- **Batch training** (not real-time serving)

A SQL view handles this perfectly.

### When Feature Store Adds Value

| Scenario | Why Feature Store Helps |
|----------|------------------------|
| **Multiple models** | Propensity, churn, LTV models share customer_tenure |
| **Training vs. serving** | Need point-in-time correctness |
| **Feature governance** | Track who created features, when, and for what |
| **Feature reuse** | Data science team shares curated features |

**Bottom line**: Feature Store adds complexity. Use it when benefits (reuse, governance) outweigh setup cost.

In [ ]:
# =============================================================================
# CELL 4: Summary
# =============================================================================

print("\n" + "="*70)
print("          SNOWFLAKE ML FEATURES - SUMMARY")
print("="*70)

print("""
WHAT WE DEMONSTRATED
--------------------

1. SQL FEATURE ENGINEERING
   - Fourier seasonality terms computed in SQL
   - Trend and fiscal flags in the view
   - Scales to any data size without Python overhead
   
2. FORECAST BASELINE  
   - Snowflake ML FORECAST for naive time-series prediction
   - 13-week forecast stored in MMM.FORECAST_BASELINE
   - Comparison view: MMM.V_FORECAST_VS_ACTUAL
   
3. FEATURE STORE GUIDANCE
   - For this demo: SQL views are sufficient
   - Feature Store adds value for multi-model, governed environments

OUTPUTS CREATED
---------------
- MMM.V_WEEKLY_REVENUE_FOR_FORECAST  - Aggregated revenue for FORECAST
- MMM.REVENUE_FORECAST_MODEL         - Snowflake ML FORECAST model
- MMM.FORECAST_BASELINE              - 13-week naive forecast

DEMO NARRATIVE
--------------
"The Snowflake FORECAST function provides a quick baseline using statistical
time-series methods. Our MMM model outperforms this because it captures the
CAUSAL impact of marketing spend, not just seasonal patterns."
""")

print("="*70)
print(f"Completed at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*70)